## About Kosarak
- Distinct element: 606770
- Number of incomming element: 990002
- Top-10 frequent element and count: 

| element | frquent |
| :----- | :----- |
| 3 | 36133 |
| 6 | 30225 |
| 6 3 | 21139 |
| 11 6 | 12769 |
| 1 | 9766 |
| 11 6 3 | 9083 |
| 1 3 | 6002 |
| 11 | 5374 |
| 1 6 | 5046 |
| 4 | 4290 |


In [3]:
# 20210316

import sys
import time
import spookyhash
import hyperloglog

def UpdateSk(e,c,width,depth):
    r=hash1(e)% depth
    ID=hash2(hash1(e))
        # compute hash value of e as ID
    found=False
        # found= True if e is in Sk[r]
    index=0
    Sk[r][0].count+=1    # total count+1
    for i in range(len(Sk[r])):
        # checks whether e is in SK[h1(e)]
        if ID == Sk[r][i].ID:
            found=True
            index=i
    if found:  # e is in Sk[r]
        Sk[r][index].count+=1 
    else:  # e is not in Sk[r]
        if len(Sk[r])<width:
            check=random.choice([0,1],size=1,p=[r/w,1-r/w])
            # append e with prob(1-r/w)
            if check:
                # put e into Sk[r]
                Sk[r].append(ID,1)
                index=len(Sk[r])-1
                found=True
        else:
            # put e into other=total count-sum(Sk[r][i].count)
            # estimate distinct element in other
            #　Sk[r][0].distinct=update(Sk[r][0].distinct,hash3(e))
            hll.add(e)
    if found:
        if Sk[r][index].count>cmax:
            emax=e
            cmax=Sk[r][index].count
    else:
        avg=(Sk[r][0].count-sum(Sk[r][i].count))/Sk[r][0].distinct
        if avg >cmax:
            emax=e
            cmax=avg
            

# main
import sys
import time

if __name__=='main':
    start=time.time()
    size=32
    Top=[size]
    Sk=[]
    item_count=100
    depth=1024
    width=4
    emax,cmax='',0
    with open("kosarak.dat",'r') as file:
        while item:
            element=file.readline().strip('\n')
            if not element:
                break
            else:
                item_count-=1
                if len(Top)==0:
                    Top.append([element,1])
                else:
                    # if e in Top
                    for i in range(len(Top)):
                        if element==Top[i][0]:
                            Top[i][1]+=1
                            found=True
                            #print("found in itemlist[{}]".format(i))
                            break
                        else:
                            found=False
                    if found:
                        pass
                    else:
                        if len(Top)<size:
                            Top.append([element,1])                        
                        else:
                            UpdateSk(element,1,width,depth)
                if cmax>Top[-1].count:
                    # bring back
                    tempe=Top[-1].ID
                    tempc=Top[-1].count
                    Top[-1].ID=emax
                    Top[-1].count=cmax
                    UpdateSk(tempe,tempc,width,depth)
    end=time.time()

print(Top)
print("Top-{} with size {} bytes.".format(len(itemlist),sys.getsizeof(itemlist)))
print("Execution time:{} seconds.".format(str(end-start)))


NameError: name 'Top' is not defined

In [1]:
# 依特定機率(1-r/w)產生1，決定e是否放到Sk[r]中
from numpy import random
import pandas as pd

w=10
print()
for r in range(w):
    li=random.choice([0,1],size=100000,p=[r/w,1-r/w])
    #print(li)
    valueCount = pd.Series(li).value_counts()
    print('valueCount/10000:', valueCount/100000)


valueCount/10000: 1    1.0
dtype: float64
valueCount/10000: 1    0.90034
0    0.09966
dtype: float64
valueCount/10000: 1    0.80013
0    0.19987
dtype: float64
valueCount/10000: 1    0.7014
0    0.2986
dtype: float64
valueCount/10000: 1    0.60032
0    0.39968
dtype: float64
valueCount/10000: 0    0.50025
1    0.49975
dtype: float64
valueCount/10000: 0    0.59725
1    0.40275
dtype: float64
valueCount/10000: 0    0.70148
1    0.29852
dtype: float64
valueCount/10000: 0    0.79992
1    0.20008
dtype: float64
valueCount/10000: 0    0.90038
1    0.09962
dtype: float64


## Hyperloglog

In [7]:
# https://github.com/svpcom/hyperloglog
import hyperloglog
import sys
import time

start=time.time()
datapath='../CountMinsketch/kosarak.dat'

hll = hyperloglog.HyperLogLog(0.01)  # accept 1% counting error

item_count=1000
with open(datapath,'r') as file:
    while True:
        element=file.readline().strip('\n')
        if not element:
            break
        else:
            # item_count-=1
            hll.add(element)
end=time.time()        
print(len(hll),type(hll),sys.getsizeof(hll))
print("Execution time:{} seconds.".format(str(end-start)))

612804 <class 'hyperloglog.hll.HyperLogLog'> 72
Execution time:6.7621448040008545 seconds.


In [20]:
# data structure
import hyperloglog


class Head():
    def __init__(self):
        self.total_count= 0
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def add_count(self,count):
        self.total_count+=count
    def sub_count(self,count):
        self.total_count-=count
    def __str__(self):
        return 'total count: {}, distinct element: {}'.format(self.total_count,len(self.distinct))
        
class Node():
    def __init__(self,ID):
        self.ID=ID
        self.count=0
    def add_count(self,count):
        self.count+=count
    def sub_count(self,count):
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
        
        

test=head()
for i in range(10):
    test.add_count(1)

print(test.total_count)
print(test)

10
total count: 10, distinct element: 0
